In [1]:
import pickle
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
OUTPUT_PATH = '../../../output_data/validation/'

In [3]:
SOT_PATH = OUTPUT_PATH + 'source_of_truth/'

In [4]:
DATA_PATH = '../../../input_data/synthetic_examples/input_data/'

In [5]:
os.listdir(SOT_PATH)

['one_line',
 'Test0a_15min_Direct_QQ.csv',
 'Test0b_15min_Direct_QQ_RT.csv',
 'test1',
 'Test1_040116_15min_Direct_QQ.csv',
 'Test1_040116_30min_Direct_QQ.csv',
 'Test1_040116_60min_Direct_QQ (2).csv',
 'Test1_040116_90min_Direct_QQ.csv',
 'test2',
 'Test2_091215_15min_Direct_QQ.csv',
 'Test2_091215_30min_Direct_QQ.csv',
 'Test2_091215_45min_Direct_QQ.csv',
 'Test2_091215_60min_Direct_QQ.csv',
 'Test2_091215_90min_Direct_QQ.csv',
 'Test3_040116_15min_Direct_QQ.csv',
 'Test3_040116_30min_Direct_QQ.csv',
 'Test3_040116_45min_Direct_QQ.csv',
 'Test3_040116_60min_Direct_QQ.csv',
 'Test3_040116_90min_Direct_QQ.csv',
 'TestParameters_Statistics.xlsx',
 'two_lines']

## Validate Two Lines : Test0b

In [7]:
os.listdir(OUTPUT_PATH + 'two_lines')

['aa_4-552.pkl',
 'all_nodes.pkl',
 'morning_direct_edges.pkl',
 'morning_nodes.pkl',
 'morning_start_nodes.pkl',
 'morning_transfer_edges.pkl',
 'nearby_stops.csv',
 'nearby_stops.pkl',
 'sa_4-552.pkl']

In [8]:
aa_tl_df = pd.read_pickle(OUTPUT_PATH + 'two_lines/aa_4-552.pkl')

In [9]:
aa_tl_df = aa_tl_df[aa_tl_df['time_sec'] <= 900]

In [10]:
aa_tl_df.shape

(791, 9)

In [11]:
sot_tl_df = pd.read_csv(SOT_PATH + 'two_lines/Test0b_15min_Direct_QQ_RT.csv')

In [12]:
sot_tl_df.shape

(1327, 7)

In [13]:
sot_tl_min_df = sot_tl_df.loc[sot_tl_df.groupby(['ID STOP START', 'ID STOP DEST'])['TRAVEL TIME'].idxmin()]

In [14]:
sot_tl_min_df.shape

(1068, 7)

In [16]:
tl_df = aa_tl_df.merge(sot_tl_min_df, left_on=['stop_id_source', 'stop_id_target'], right_on=['ID STOP START', 'ID STOP DEST'])

In [17]:
tl_df.shape

(733, 16)

In [18]:
tl_df.head()

,stop_id_source,stop_id_target,time_sec,stop_lon_target,stop_lat_target,arrival_target,stop_lon_source,stop_lat_source,departure,ID STOP START,TIME DEPART,ID STOP DEST,TIME ARRIVAL,TRAVEL TIME,ORG_ROUTE_ID,DEST_ROUTE_ID
0,12816,12816,0.0,34.821988,32.059675,2015-12-09 07:28:16,34.821988,32.059675,2015-12-09 07:28:16,12816,07:28:16,12816,07:28:16,0,2517,2517
1,12816,12837,127.0,34.828604,32.058354,2015-12-09 07:20:23,34.821988,32.059675,2015-12-09 07:18:16,12816,07:28:16,12837,07:30:23,127,2517,2517
2,12816,12867,304.0,34.833911,32.055035,2015-12-09 07:23:20,34.821988,32.059675,2015-12-09 07:18:16,12816,07:28:16,12867,07:33:20,304,2517,2517
3,12837,12837,0.0,34.828604,32.058354,2015-12-09 07:20:23,34.828604,32.058354,2015-12-09 07:20:23,12837,07:20:23,12837,07:20:23,0,2517,2517
4,12837,12867,177.0,34.833911,32.055035,2015-12-09 07:23:20,34.828604,32.058354,2015-12-09 07:20:23,12837,07:20:23,12867,07:23:20,177,2517,2517


In [19]:
tl_df['diff'] = tl_df['time_sec'] - tl_df['TRAVEL TIME']

In [20]:
tl_df[tl_df['diff'] != 0]

,stop_id_source,stop_id_target,time_sec,stop_lon_target,stop_lat_target,arrival_target,stop_lon_source,stop_lat_source,departure,ID STOP START,TIME DEPART,ID STOP DEST,TIME ARRIVAL,TRAVEL TIME,ORG_ROUTE_ID,DEST_ROUTE_ID,diff


### We don't have any stop-to-stop trips that have any difference in computed times

### Check what are the trips that appear in the source of truth but not in my computations (source-target pairs)

In [22]:
t1_outer_join = aa_tl_df.merge(sot_tl_min_df, left_on=['stop_id_source', 'stop_id_target'], right_on=['ID STOP START', 'ID STOP DEST'], how='outer')

In [23]:
t1_outer_join.shape

(1126, 16)

In [24]:
trips_not_in_my_computations_df = t1_outer_join[t1_outer_join['departure'].isna()]

In [25]:
trips_not_in_my_computations_df.shape

(335, 16)

In [26]:
trips_not_in_my_computations_df.head()

,stop_id_source,stop_id_target,time_sec,stop_lon_target,stop_lat_target,arrival_target,stop_lon_source,stop_lat_source,departure,ID STOP START,TIME DEPART,ID STOP DEST,TIME ARRIVAL,TRAVEL TIME,ORG_ROUTE_ID,DEST_ROUTE_ID
791,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,12843.0,07:24:31,12837.0,07:40:23,952.0,2517.0,2517.0
792,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,12843.0,07:24:31,12867.0,07:43:20,1129.0,2517.0,2517.0
793,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,12844.0,07:24:08,12837.0,07:40:23,975.0,2517.0,2517.0
794,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,12844.0,07:24:08,12867.0,07:43:20,1152.0,2517.0,2517.0
795,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,12882.0,07:28:33,13266.0,07:44:41,968.0,9823.0,9823.0


In [27]:
# Remove rows where source != target and where travel time is more than 900 seconds (15 minutes)
trips_not_in_my_computations_df = trips_not_in_my_computations_df[trips_not_in_my_computations_df['ID STOP START'] != trips_not_in_my_computations_df['ID STOP DEST']]
trips_not_in_my_computations_df = trips_not_in_my_computations_df[trips_not_in_my_computations_df['TRAVEL TIME'] <= 900]

In [28]:
trips_not_in_my_computations_df.shape

(0, 16)

In [29]:
trips_not_in_my_computations_df.head()

,stop_id_source,stop_id_target,time_sec,stop_lon_target,stop_lat_target,arrival_target,stop_lon_source,stop_lat_source,departure,ID STOP START,TIME DEPART,ID STOP DEST,TIME ARRIVAL,TRAVEL TIME,ORG_ROUTE_ID,DEST_ROUTE_ID
